In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [3]:
class minmax():
    def __init__(self):
        pass
        
    def minmax_transform_fit(self, train_df, valid_df):

        scaler = MinMaxScaler()
        scaler = scaler.fit(train_df)

        train_scaled = scaler.transform(train_df)
        train_scaled = pd.DataFrame(train_scaled)

        valid_scaled = scaler.transform(valid_df)
        valid_scaled = pd.DataFrame(valid_scaled)

        return scaler, train_scaled, valid_scaled
    
    def minmax_transform(self, scaler, test_df):
        
        test_scaled = scaler.transform(test_df)
        test_scaled = pd.DataFrame(test_scaled)

        return test_scaled
    
    def inverse_minmax(self, scaler, df):
        return scaler.inverse_transform(df)

In [4]:
class myPCA():
    def __init__(self):
        pass

    def PCA_transform_fit(self, train_df, valid_df):
        n = len(train_df.columns)
        self.pca = PCA(n_components = n)
        pca_train = self.pca.fit_transform(train_df)
        self.pca_train_df = pd.DataFrame(pca_train, columns = [f"pca{num + 1}" for num in range(n)])

        eig = self.pca.explained_variance_
        ratio = self.pca.explained_variance_ratio_
        acc_ratio = ratio.cumsum()

        self.pca_info = pd.DataFrame({
            "eig for variance": eig,
            "acc. explained": acc_ratio},
            index = np.array([f"pca{num + 1}" for num in range(n)])
        )

        pca_valid = self.pca.transform(valid_df)
        self.pca_valid_df = pd.DataFrame(pca_valid, columns = [f"pca{num + 1}" for num in range(n)])

        return self.pca_train_df, self.pca_valid_df
    
    def PCA_transform(self, test_df):
        
        n = len(test_df.columns)
        pca_test = self.pca.transform(test_df)
        self.pca_test_df = pd.DataFrame(pca_test, columns = [f"pca{num + 1}" for num in range(n)])
        
        return self.pca_test_df

# eigenvalue for variance >= 0.70
# accumulated ratio of variance explained >= 0.80

    def set_params(self):
        for i in range(len(self.pca_train_df.columns)):
            if (self.pca_info["acc. explained"][i] >= 0.80):
                self.param = i+1
                return self.param
        return len(self.pca_train_df.columns)-1    

In [5]:
class Preprocessing_minmax:
    
    matching_dict = {"1018640" : ["402", "403", "413"], "1018662": ["403", "413", "421"], "1018680": ["415", "510", "889"], "1018683": ["415", "510", "889"]}
    
    def __init__(self, code): #my_type: 0 (new fitting ... cv_num = 7, need to split test and train set) 1 (final fitting ... cv_num = 1, don't need to split test and train set)
        # code: water level region code
        # comp: component made by mstl
        self.code = code
        #self.my_type = my_type
        # if my_type == 0:
        #     self.num = 8 # blocking_cv num ... considering test set, set cv_num 8.
        # elif my_type == 1:
        #     self.num = 1
        
    def merging(self):
        
        self.loc_code = self.matching_dict[self.code]
        
        resid_lvl = pd.read_csv(f"./data_MSTL/MSTLtest_lvl_{self.code}_resid.csv")
        #se24_lvl = pd.read_csv(f"./data_MSTL/MSTL3_lvl_{self.code}_se24.csv")
        #se360_lvl = pd.read_csv(f"./data_MSTL/MSTL3_lvl_{self.code}_se360.csv")
        se8760_lvl = pd.read_csv(f"./data_MSTL/MSTLtest_lvl_{self.code}_se8760.csv")
        trend_lvl = pd.read_csv(f"./data_MSTL/MSTLtest_lvl_{self.code}_trend.csv")
        
        resid_fir = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[0]}_resid.csv")
        #se24_fir = pd.read_csv(f"./data_MSTL/MSTL3_climate_{self.loc_code[0]}_se24.csv")
        #se360_fir = pd.read_csv(f"./data_MSTL/MSTL3_climate_{self.loc_code[0]}_se360.csv")
        se8760_fir = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[0]}_se8760.csv")
        trend_fir = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[0]}_trend.csv")
        
        resid_sec = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[1]}_resid.csv")
        #se24_sec = pd.read_csv(f"./data_MSTL/MSTL3_climate_{self.loc_code[1]}_se24.csv")
        #se360_sec = pd.read_csv(f"./data_MSTL/MSTL_climate_{self.loc_code[1]}_se360.csv")
        se8760_sec = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[1]}_se8760.csv")
        trend_sec = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[1]}_trend.csv")
        
        resid_thr = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[2]}_resid.csv")
        #se24_thr = pd.read_csv(f"./data_MSTL/MSTL3_climate_{self.loc_code[2]}_se24.csv")
        #se360_thr = pd.read_csv(f"./data_MSTL/MSTL3_climate_{self.loc_code[2]}_se360.csv")
        se8760_thr = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[2]}_se8760.csv")
        trend_thr = pd.read_csv(f"./data_MSTL/MSTLtest_climate_{self.loc_code[2]}_trend.csv")
        
        resid_lvl.drop("Unnamed: 0", inplace = True, axis = 1)
        #se24_lvl.drop("Unnamed: 0", inplace = True, axis = 1)
        #se360_lvl.drop("Unnamed: 0", inplace = True, axis = 1)
        se8760_lvl.drop("Unnamed: 0", inplace = True, axis = 1)
        trend_lvl.drop("Unnamed: 0", inplace = True, axis = 1)
        
        resid_fir.drop("Unnamed: 0", inplace = True, axis = 1)
        # se24_fir.drop("Unnamed: 0", inplace = True, axis = 1)
        #se360_fir.drop("Unnamed: 0", inplace = True, axis = 1)
        se8760_fir.drop("Unnamed: 0", inplace = True, axis = 1)
        trend_fir.drop("Unnamed: 0", inplace = True, axis = 1)
        
        resid_sec.drop("Unnamed: 0", inplace = True, axis = 1)
        # se24_sec.drop("Unnamed: 0", inplace = True, axis = 1)
        #se360_sec.drop("Unnamed: 0", inplace = True, axis = 1)
        se8760_sec.drop("Unnamed: 0", inplace = True, axis = 1)
        trend_sec.drop("Unnamed: 0", inplace = True, axis = 1)
        
        resid_thr.drop("Unnamed: 0", inplace = True, axis = 1)
        # se24_thr.drop("Unnamed: 0", inplace = True, axis = 1)
        #se360_thr.drop("Unnamed: 0", inplace = True, axis = 1)
        se8760_thr.drop("Unnamed: 0", inplace = True, axis = 1)
        trend_thr.drop("Unnamed: 0", inplace = True, axis = 1)
        
        resid_lvl.columns = ["lvl_re"]
        # se24_lvl.columns = ["lvl_se24"]
        #se360_lvl.columns = ["lvl_se360"]
        se8760_lvl.columns = ["lvl_se8760"]
        trend_lvl.columns = ["lvl_tr"]
        
        #print(resid_fir)
        print(self.loc_code[0])
        
        resid_fir.columns = ["fir_re_temp", "fir_re_pre", "fir_re_ver", "fir_re_hor"]
        # se24_fir.columns = ["fir_se24_temp", "fir_se24_pre", "fir_se24_ver", "fir_se24_hor"]
        #se360_fir.columns = ["fir_se360_temp", "fir_se360_pre", "fir_se360_ver", "fir_se360_hor"]
        se8760_fir.columns = ["fir_se8760_temp", "fir_se8760_pre", "fir_se8760_ver", "fir_se8760_hor"]
        trend_fir.columns = ["fir_tr_temp", "fir_tr_pre", "fir_tr_ver", "fir_tr_hor"]
        
        resid_sec.columns = ["sec_re_temp", "sec_re_pre", "sec_re_ver", "sec_re_hor"]
        # se24_sec.columns = ["sec_se24_temp", "sec_se24_pre", "sec_se24_ver", "sec_se24_hor"]
        #se360_sec.columns = ["sec_se360_temp", "sec_se360_pre", "sec_se360_ver", "sec_se360_hor"]
        se8760_sec.columns = ["sec_se8760_temp", "sec_se8760_pre", "sec_se8760_ver", "sec_se8760_hor"]
        trend_sec.columns = ["sec_tr_temp", "sec_tr_pre", "sec_tr_ver", "sec_tr_hor"]
        
        resid_thr.columns = ["thr_re_temp", "thr_re_pre", "thr_re_ver", "thr_re_hor"]
        # se24_thr.columns = ["thr_se24_temp", "thr_se24_pre", "thr_se24_ver", "thr_se24_hor"]
        #se360_thr.columns = ["thr_se360_temp", "thr_se360_pre", "thr_se360_ver", "thr_se360_hor"]
        se8760_thr.columns = ["thr_se8760_temp", "thr_se8760_pre", "thr_se8760_ver", "thr_se8760_hor"]
        trend_thr.columns = ["thr_tr_temp", "thr_tr_pre", "thr_tr_ver", "thr_tr_hor"]
        
        
        # self.merged_df = pd.concat([resid_lvl, se24_lvl, se360_lvl, se8760_lvl, trend_lvl,
        #                            resid_fir, se24_fir, se360_fir, se8760_fir, trend_fir,
        #                            resid_sec, se24_sec, se360_sec, se8760_sec, trend_sec,
        #                            resid_thr, se24_thr, se360_thr, se8760_thr, trend_thr], axis = 1)
            
        self.merged_df = pd.concat([resid_lvl, se8760_lvl, trend_lvl,
                                   resid_fir, se8760_fir, trend_fir,
                                   resid_sec, se8760_sec, trend_sec,
                                   resid_thr, se8760_thr, trend_thr], axis = 1)
        
        self.merged_df = self.merged_df.dropna()

        #merged_df.head(30)
        
    def make_ind_set(self, num):
        blocking_instance = BlockingTimeSeriesSplit(n_splits = num)
        split_data = blocking_instance.split(self.merged_df)
        ind_set = []
        for i in range(num):
            ind_set += next(split_data)
        return ind_set
        
    def do_blocking(self):
        (self.full_data, self.full_data_test) = self.blocking(8)
        (self.full_data_final_test, _) = self.blocking(1)
        
    def do_minmax_and_pca(self):
        self.minmax_and_pca(8)
        
    def blocking(self, num):
        
        ind_set = self.make_ind_set(num)
        
        # trend = []; seasonal_24 = []; seasonal_360 = []; seasonal_8760 = []; residual = []
        trend = []; seasonal_8760 = []; residual = []
        trend_test = []; seasonal_8760_test = []; residual_test = []

        for i, ind in enumerate(ind_set):
            if i%2 == 0:
                train_df = self.merged_df.loc[ind]
                train_df.reset_index()
                # x_train = train_df.drop(['lvl_tr', 'lvl_se24', 'lvl_se360', 'lvl_se8760', 'lvl_re'], axis=1)
                x_train = train_df.drop(['lvl_tr', 'lvl_se8760', 'lvl_re'], axis=1)
            
                x_train_tr = train_df[[
                    "fir_tr_temp", "fir_tr_pre", "fir_tr_ver", "fir_tr_hor",
                    "sec_tr_temp", "sec_tr_pre", "sec_tr_ver", "sec_tr_hor",
                    "thr_tr_temp", "thr_tr_pre", "thr_tr_ver", "thr_tr_hor"
                ]]
                # x_train_se24 = train_df[[
                #     "fir_se24_temp", "fir_se24_pre", "fir_se24_ver", "fir_se24_hor",
                #     "sec_se24_temp", "sec_se24_pre", "sec_se24_ver", "sec_se24_hor",
                #     "thr_se24_temp", "thr_se24_pre", "thr_se24_ver", "thr_se24_hor"
                # ]]
                # x_train_se360 = train_df[[
                #     "fir_se360_temp", "fir_se360_pre", "fir_se360_ver", "fir_se360_hor",
                #     "sec_se360_temp", "sec_se360_pre", "sec_se360_ver", "sec_se360_hor",
                #     "thr_se360_temp", "thr_se360_pre", "thr_se360_ver", "thr_se360_hor"
                # ]]
                x_train_se8760 = train_df[[
                    "fir_se8760_temp", "fir_se8760_pre", "fir_se8760_ver", "fir_se8760_hor",
                    "sec_se8760_temp", "sec_se8760_pre", "sec_se8760_ver", "sec_se8760_hor",
                    "thr_se8760_temp", "thr_se8760_pre", "thr_se8760_ver", "thr_se8760_hor"
                ]]
                x_train_re = train_df[[
                    "fir_re_temp", "fir_re_pre", "fir_re_ver", "fir_re_hor",
                    "sec_re_temp", "sec_re_pre", "sec_re_ver", "sec_re_hor",
                    "thr_re_temp", "thr_re_pre", "thr_re_ver", "thr_re_hor"
                ]]
                y_train_tr = train_df[['lvl_tr']]
                # y_train_se24 = train_df[['lvl_se24']]
                # y_train_se360 = train_df[['lvl_se360']]
                y_train_se8760 = train_df[['lvl_se8760']]
                y_train_re = train_df[['lvl_re']]
            else:
                valid_df = self.merged_df.loc[ind]
                valid_df.reset_index()
                
                #x_valid = valid_df.drop(['lvl_tr', 'lvl_se24', 'lvl_se360', 'lvl_se8760', 'lvl_re'], axis=1)
                x_valid = valid_df.drop(['lvl_tr', 'lvl_se8760', 'lvl_re'], axis=1)
                
                x_valid_tr = valid_df[[
                    "fir_tr_temp", "fir_tr_pre", "fir_tr_ver", "fir_tr_hor",
                    "sec_tr_temp", "sec_tr_pre", "sec_tr_ver", "sec_tr_hor",
                    "thr_tr_temp", "thr_tr_pre", "thr_tr_ver", "thr_tr_hor"
                ]]
                # x_valid_se24 = valid_df[[
                #     "fir_se24_temp", "fir_se24_pre", "fir_se24_ver", "fir_se24_hor",
                #     "sec_se24_temp", "sec_se24_pre", "sec_se24_ver", "sec_se24_hor",
                #     "thr_se24_temp", "thr_se24_pre", "thr_se24_ver", "thr_se24_hor"
                # ]]
                # x_valid_se360 = valid_df[[
                #     "fir_se360_temp", "fir_se360_pre", "fir_se360_ver", "fir_se360_hor",
                #     "sec_se360_temp", "sec_se360_pre", "sec_se360_ver", "sec_se360_hor",
                #     "thr_se360_temp", "thr_se360_pre", "thr_se360_ver", "thr_se360_hor"
                # ]]
                x_valid_se8760 = valid_df[[
                    "fir_se8760_temp", "fir_se8760_pre", "fir_se8760_ver", "fir_se8760_hor",
                    "sec_se8760_temp", "sec_se8760_pre", "sec_se8760_ver", "sec_se8760_hor",
                    "thr_se8760_temp", "thr_se8760_pre", "thr_se8760_ver", "thr_se8760_hor"
                ]]
                x_valid_re = valid_df[[
                    "fir_re_temp", "fir_re_pre", "fir_re_ver", "fir_re_hor",
                    "sec_re_temp", "sec_re_pre", "sec_re_ver", "sec_re_hor",
                    "thr_re_temp", "thr_re_pre", "thr_re_ver", "thr_re_hor"
                ]]
                y_valid_tr = valid_df[['lvl_tr']]
                # y_valid_se24 = valid_df[['lvl_se24']]
                # y_valid_se360 = valid_df[['lvl_se360']]
                y_valid_se8760 = valid_df[['lvl_se8760']]
                y_valid_re = valid_df[['lvl_re']]
                
                if (i == 2 * num - 1 and i > 1):
                    x_test_tr = pd.concat([x_train_tr, x_valid_tr], ignore_index=True)
                    y_test_tr = pd.concat([y_train_tr, y_valid_tr], ignore_index=True)
                    x_test_se8760 = pd.concat([x_train_se8760, x_valid_se8760], ignore_index=True)
                    y_test_se8760 = pd.concat([y_train_se8760, y_valid_se8760], ignore_index=True)
                    x_test_re = pd.concat([x_train_re, x_valid_re], ignore_index=True)
                    y_test_re = pd.concat([y_train_re, y_valid_re], ignore_index=True)
                    trend_test.append([x_test_tr, y_test_tr])
                    seasonal_8760_test.append([x_test_se8760, y_test_se8760])
                    residual_test.append([x_test_re, y_test_re])
                    
                else:
                    trend.append([x_train_tr, y_train_tr, x_valid_tr, y_valid_tr])
                    # seasonal_24.append([x_train_se24, y_train_se24, x_valid_se24, y_valid_se24])
                    # seasonal_360.append([x_train_se360, y_train_se360, x_valid_se360, y_valid_se360])
                    seasonal_8760.append([x_train_se8760, y_train_se8760, x_valid_se8760, y_valid_se8760])
                    residual.append([x_train_re, y_train_re, x_valid_re, y_valid_re]) 
        
        # self.full_data = [trend, seasonal_24, seasonal_360, seasonal_8760, residual]
        full_data = [trend, seasonal_8760, residual]
        full_data_test = [trend_test, seasonal_8760_test, residual_test]
        
        return (full_data, full_data_test)
        
        
        #self.full_data_final_test = [trend_test, seasonal_8760_test, residual_test]
    
    def minmax_and_pca(self, num):
        
        self.pca_full_data = self.full_data
        self.pca_full_data_test = self.full_data_test
        self.pca_full_data_final_test = self.full_data_final_test
        
        self.minmax = minmax()
        
        self.scaler_X = []
        self.scaler_y = []
        
        ind_set = self.make_ind_set(num)[:-2]
        
        for i in range(len(self.full_data)):
            
            input_df0 = pd.DataFrame([])
            input_df1 = pd.DataFrame([])
            input_df2 = pd.DataFrame([])
            input_df3 = pd.DataFrame([])
            
            input_df0_test = pd.DataFrame([])
            input_df1_test = pd.DataFrame([])
            
            input_df0_final_test = pd.DataFrame([])
            input_df1_final_test = pd.DataFrame([])
            input_df2_final_test = pd.DataFrame([])
            input_df3_final_test = pd.DataFrame([])
            
            for j in range(len(self.full_data[0])):
                input_df0 = pd.concat([input_df0, self.full_data[i][j][0]]) # x train
                input_df1 = pd.concat([input_df1, self.full_data[i][j][1]]) # y train
                input_df2 = pd.concat([input_df2, self.full_data[i][j][2]]) # x valid
                input_df3 = pd.concat([input_df3, self.full_data[i][j][3]]) # y valid
                
            input_df0_test = self.full_data_test[i][0][0] # x test
            input_df1_test =self.full_data_test[i][0][1] # y test
                
            input_df0_final_test = self.full_data_final_test[i][0][0] 
            input_df1_final_test = self.full_data_final_test[i][0][1] 
            input_df2_final_test = self.full_data_final_test[i][0][2] 
            input_df3_final_test = self.full_data_final_test[i][0][3] 
            
            (scaler, df0, df2) = self.minmax.minmax_transform_fit(input_df0, input_df2) # x
            self.scaler_X.append(scaler)
            
            df0_test = self.minmax.minmax_transform(scaler, input_df0_test)
            df0_final_test = self.minmax.minmax_transform(scaler, input_df0_final_test)
            df2_final_test = self.minmax.minmax_transform(scaler, input_df2_final_test)
            
            (scaler, df1, df3) = self.minmax.minmax_transform_fit(input_df1, input_df3) # y
            self.scaler_y.append(scaler)
            
            df1_test = self.minmax.minmax_transform(scaler, input_df1_test)
            df1_final_test = self.minmax.minmax_transform(scaler, input_df1_final_test)
            df3_final_test = self.minmax.minmax_transform(scaler, input_df3_final_test)
            
            
            
            self.pca_instance = myPCA()
            x_train_pca, x_valid_pca = self.pca_instance.PCA_transform_fit(df0, df2)
            no = self.pca_instance.set_params()
            x_train_pca, x_valid_pca = x_train_pca[[f"pca{d+1}" for d in range(no)]], x_valid_pca[[f"pca{d+1}" for d in range(no)]]
            y_train_pca, y_valid_pca = (df1, df3)
            
            x_test_pca = self.pca_instance.PCA_transform(df0_test)
            x_test_pca = x_test_pca[[f"pca{d+1}" for d in range(no)]]
            x_final_train_pca = self.pca_instance.PCA_transform(df0_final_test)
            x_final_train_pca = x_final_train_pca[[f"pca{d+1}" for d in range(no)]]
            x_final_valid_pca = self.pca_instance.PCA_transform(df2_final_test)
            x_final_valid_pca = x_final_valid_pca[[f"pca{d+1}" for d in range(no)]]
            
            y_test_pca = df1_test
            y_final_train_pca = df1_final_test
            y_final_valid_pca = df3_final_test

            delta1 = len(ind_set[0])
            delta2 = len(ind_set[1])
            
            for k, ind in enumerate(ind_set):
                if k % 2 == 0:
                    self.pca_full_data[i][k//2][0] = x_train_pca.loc[[ind[i] - delta2 * k//2 for i in range(len(ind))]] # x train
                    self.pca_full_data[i][k//2][0].reset_index(inplace=True, drop = True)
                    self.pca_full_data[i][k//2][1] = y_train_pca.loc[[ind[i] - delta2 * k//2 for i in range(len(ind))]] # y train
                    self.pca_full_data[i][k//2][1].reset_index(inplace=True, drop = True)
                else:
                    self.pca_full_data[i][k//2][2] = x_valid_pca.loc[[ind[i] - delta1 * (k//2 + 1) for i in range(len(ind))]] # x valid
                    self.pca_full_data[i][k//2][2].reset_index(inplace=True, drop = True)
                    self.pca_full_data[i][k//2][3] = y_valid_pca.loc[[ind[i] - delta1 * (k//2 + 1) for i in range(len(ind))]] # y valid 
                    self.pca_full_data[i][k//2][3].reset_index(inplace=True, drop = True)
        
            self.pca_full_data_test[i][0][0] = x_test_pca
            self.pca_full_data_test[i][0][0].reset_index(inplace=True, drop = True)
            self.pca_full_data_test[i][0][1] = y_test_pca
            self.pca_full_data_test[i][0][1].reset_index(inplace=True, drop = True)
            
            self.pca_full_data_final_test[i][0][0] = x_final_train_pca
            self.pca_full_data_final_test[i][0][0].reset_index(inplace=True, drop = True)
            self.pca_full_data_final_test[i][0][1] = y_final_train_pca
            self.pca_full_data_final_test[i][0][1].reset_index(inplace=True, drop = True)
            self.pca_full_data_final_test[i][0][2] = x_final_valid_pca
            self.pca_full_data_final_test[i][0][2].reset_index(inplace=True, drop = True)
            self.pca_full_data_final_test[i][0][3] = y_final_valid_pca
            self.pca_full_data_final_test[i][0][3].reset_index(inplace=True, drop = True)
            
            
            
            
            # (scaler, df0, df2) = self.normalization.normalize_transform_fit(input_df0, input_df2) # x
            # self.scaler_X.append(scaler)
            # (scaler, df1, df3) = self.normalization.normalize_transform_fit(input_df1, input_df3) # y
            # self.scaler_y.append(scaler)
            
            #x_test_pca = self.pca_instance.PCA_transform(df2)
            #x_train_pca, x_valid_pca = x_train_pca[[f"pca{d+1}" for d in range(no)]], x_valid_pca[[f"pca{d+1}" for d in range(no)]] 
                
                
                #self.pca_full_data[i].append([x_train_pca, y_train_pca, x_valid_pca, y_valid_pca])
        
        
                 
            # # for j in range(len(self.full_data[0])):
            # #     (scaler, self.scaled_data[i][j][0], self.scaled_data[i][j][2]) = self.normalization.normalize(self.full_data[i][j][0], self.full_data[i][j][2])
            # #     temp_X.append(scaler)
            # #     (scaler, self.scaled_data[i][j][1], self.scaled_data[i][j][3]) = self.normalization.normalize(self.full_data[i][j][1], self.full_data[i][j][3])
            # #     temp_y.append(scaler)
            # self.scaler_X.append(temp_X)
            # self.scaler_y.append(temp_y)
            
            
    def inverse_minmax(self, pred): # df may be a prediction vector (3d array)
        
        return_pred = pred
        
        for i in range(3):
            return_pred[i] = self.minmax.inverse_minmax(self.scaler_y[i], pred[i].reshape(1, -1))
        #print(pred[0][0], return_pred[0][0])
        return return_pred 
        
#     def pca(self):
        
        
#         self.scaled_data = self.full_data
        
#         self.normalization = normalization()
        
#         self.scaler_X = []
#         self.scaler_y = []
        
#         ind_set = self.make_ind_set(num)[:-2]
#         print(ind_set)
        
#         for i in range(len(self.full_data)):
            
#             input_df0 = pd.DataFrame([])
#             input_df1 = pd.DataFrame([])
#             input_df2 = pd.DataFrame([])
#             input_df3 = pd.DataFrame([])
#             for j in range(len(self.full_data[0])):
#                 input_df0 = pd.concat([input_df0, self.full_data[i][j][0]]) # x train
#                 input_df1 = pd.concat([input_df1, self.full_data[i][j][1]]) # y train
#                 input_df2 = pd.concat([input_df2, self.full_data[i][j][2]]) # x valid
#                 input_df3 = pd.concat([input_df3, self.full_data[i][j][3]]) # y valid
            
#             (scaler, df0, df2) = self.normalization.normalize(input_df0, input_df2) # x
#             self.scaler_X.append(scaler)
#             (scaler, df1, df3) = self.normalization.normalize(input_df1, input_df3) # y
#             self.scaler_y.append(scaler)
            
#         pca_tr = []; pca_sn8760 = []; pca_re = []
#         self.pca_full_data = [pca_tr, pca_sn8760, pca_re]
        
        
        
#         # pca_tr = []; pca_sn24 = []; pca_sn360 = []; pca_sn8760 = []; pca_re = []
#         pca_tr = []; pca_sn8760 = []; pca_re = []
#         #self.pca_full_data = [pca_tr, pca_sn24, pca_sn360, pca_sn8760, pca_re]
#         self.pca_full_data = [pca_tr, pca_sn8760, pca_re]

#         for i, comp in enumerate(self.scaled_data):
#             for split in range(self.num):
#                 self.pca_instance = myPCA()
#                 x_train_pca, x_valid_pca = self.pca_instance.result_PCA(comp[split][0], comp[split][2])
#                 no = self.pca_instance.set_params()
#                 x_train_pca, x_valid_pca = x_train_pca[[f"pca{d+1}" for d in range(no)]], x_valid_pca[[f"pca{d+1}" for d in range(no)]]
#                 y_train_pca, y_valid_pca = (comp[split][1], comp[split][3])
#                 self.pca_full_data[i].append([x_train_pca, y_train_pca, x_valid_pca, y_valid_pca])
            
#         return self.pca_full_data

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

from sklearn.metrics import mean_squared_error 

import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

import tensorflow as tf
from tensorflow import keras

# from pyFTS.common import Transformations, Util as cUtil, Membership as mf
# from pyFTS.benchmarks import benchmarks as bchmk, Util as bUtil, Measures
# from pyFTS.partitioners import CMeans, Grid, FCM, Huarng, Entropy, Util as pUtil
# from pyFTS.models.multivariate import common, variable, mvfts
# from pyFTS.models import hofts
# from pyFTS.models import chen

import seaborn as sn

import math

from joblib import dump, load

2023-10-22 07:41:25.692368: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [15]:
class predict_model:
    
    def __init__(self, df, code):
        self.type_dict = {'trend': 0, 'seasonal': 1, 'residual': 2}
        
        self.dataset = df
        self.code = code # 수위 예측 지점 코드
        
        self.trend_model = 0
        
        self.scaler_X = []
        self.scaler_y = []
        self.seasonal_model = 0
        
        self.df = []
    
    @staticmethod
    def SMAPE(y_test, y_pred):
        y_test = y_test.to_numpy(); y_pred = y_pred.to_numpy()
        return np.mean((np.abs(y_test-y_pred))/(np.abs(y_test)+np.abs(y_pred)))*100

    @staticmethod
    def RMSE(y_test, y_pred):
        return np.sqrt(mean_squared_error(y_test, y_pred))
    
    @staticmethod
    def SMAPE_np(y_test, y_pred):
        return np.mean((np.abs(y_test-y_pred))/(np.abs(y_test)+np.abs(y_pred)))*100
    
    @staticmethod
    def RMSE_np(y_test, y_pred):
        return np.mean((y_test - y_pred)**2)
        
    def fit_VAR_model(self, my_type, is_final, lag = 15, step = 3):
        given_data = self.dataset[self.type_dict[my_type]] # trend
        
        least_rmse = 1000

        for idx, block_data in enumerate(given_data):
            X_train = block_data[0]
            y_train = block_data[1]
            X_valid = block_data[2]
            y_valid = block_data[3]
            
            SMAPE_sum = 0
            RMSE_sum = 0
            
            df_train = pd.merge(y_train, X_train, left_index=True, right_index=True, how='inner')
            df_valid = pd.merge(y_valid, X_valid, left_index=True, right_index=True, how='inner')

            print(f"Modeling {my_type} component... SPLIT %d"%idx, end = "\r")
            premodel = VAR(df_train)
            model = premodel.fit(lag)

            df_valid = pd.concat([df_train.loc[-lag: , :], df_valid])
            

            for i in range(0, len(df_valid)-lag-step):
                lagged_values = df_valid.values[i: i+lag]
                forecast = pd.DataFrame(model.forecast(y = lagged_values, steps = step), columns=df_train.columns)
                SMAPE_sum += self.SMAPE(forecast.iloc[:step, [0]], df_valid.iloc[i+lag: i+step+lag, [0]])
                RMSE_sum += self.RMSE(forecast.iloc[:step, [0]], df_valid.iloc[i+lag: i+step+lag, [0]]) 
            
            print(f"TREND SPLIT {idx}: SMAPE avg {SMAPE_sum / (len(df_valid) - lag)}, RMSE avg {RMSE_sum / (len(df_valid) - lag)}")
            
            if least_rmse > RMSE_sum / (len(df_valid) - lag):
                least_rmse = RMSE_sum / (len(df_valid) - lag)
                best_model = model
                
        if is_final == 1:
            dump(best_model, f'{my_type}_{self.code}_model_final.joblib')
        else:
            dump(best_model, f'{my_type}_{self.code}_model.joblib')
        
    def eval_VAR_model(self, df, my_type):
        
        lag = 15; step = 3
        
        model = load(f'{my_type}_{self.code}_model.joblib')
        
        given_data = df[self.type_dict[my_type]]
        
        X_test = given_data[0][0]
        y_test = given_data[0][1]
        
        df_test = pd.merge(y_test, X_test, left_index=True, right_index=True, how='inner')
        
        pred = []
        
        for i in range(0, len(df_test)-lag-step):
            lagged_values = df_test.values[i: i+lag]
            forecast = pd.DataFrame(model.forecast(y = lagged_values, steps = step), columns=df_test.columns)
            pred.append([[forecast[0][0]], [forecast[0][1]], [forecast[0][2]]])
            
        return np.array(pred)


    def pred_VAR_model(self, my_type):
        
        df = self.dataset
        
        lag = 15; step = 3
        
        model = load(f'{my_type}_{self.code}_model_final.joblib')
        
        given_data = df[0]
        
        X_test = given_data[0][0]
        y_test = given_data[0][1]
        
        df_test = pd.merge(y_test, X_test, left_index=True, right_index=True, how='inner')
        
        pred = []
        
        i = len(df_test)-lag
        lagged_values = df_test.values[i: i+lag]
        forecast = pd.DataFrame(model.forecast(y = lagged_values, steps = step), columns=df_test.columns)
        pred.append([[forecast[0][0]], [forecast[0][1]], [forecast[0][2]]])
            
        return np.array(pred)
        
    ###################         

    def make_dataset(self, X_df, y_df, time_step = 30, y_step = 3):
        new_X_df = []
        new_y_df = []
        X_df = pd.concat([X_df, y_df], axis=1)
        for i in range(len(X_df) - time_step - y_step + 1):
            new_X_df.append(np.array(X_df.iloc[i:i + time_step])) # i ~ i + time_step - 1
            new_y_df.append(np.array(y_df.iloc[i + time_step: i + time_step + y_step])) # i + time_step ~ i + time_step + y_step - 1
        return np.array(new_X_df), np.array(new_y_df)
    
    
    def calculate_metrics(self, true, pred):
        true = pd.DataFrame(true)
        pred = pd.DataFrame(pred)
        mae = (true - pred).abs().mean()
        mape = (true - pred).abs().div(true).mean() * 100
        rmse = np.sqrt(((true - pred) ** 2).mean())
        return {
            "mae": mae,
            "mape": mape,
            "rmse": rmse,
        }
    
    def Bi_Seq2_model(self, feature_num, time_step, y_step):

        model = keras.Sequential()
        model.add(keras.layers.Bidirectional(keras.layers.LSTM(128, activation = 'tanh', input_shape = (time_step, feature_num), return_sequences = True, kernel_initializer='he_normal')))
        model.add(keras.layers.Bidirectional(keras.layers.LSTM(64, activation = 'tanh')))
        model.add(keras.layers.RepeatVector(y_step))
        model.add(keras.layers.LSTM(64, activation = 'tanh', return_sequences = True))
        model.add(keras.layers.LSTM(128, activation = 'tanh', return_sequences = True))
        model.add(keras.layers.TimeDistributed(keras.layers.Dense(1)))
        #model.compile(optimizer = 'nadam', loss = 'mse')
        #model.summary()

        return model
    
    def Bi_Seq2_fit(self, model, X_train, y_train, X_valid, y_valid, time_step = 8, patience = 50, epochs = 1000):

        model.compile(loss='mean_squared_error', optimizer = 'nadam')
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience = patience)

        history = model.fit(X_train, y_train, 
                            epochs = epochs,
                            validation_data = (X_valid, y_valid), 
                            callbacks = [early_stop],
                            shuffle = False)
        return history
    
    def fit_Bi_Seq2(self, my_type, is_final, time_step = 30, y_step = 3, patience = 100, epochs = 1000):
        
        self.scaled_data = self.dataset[self.type_dict[my_type]]
        
        pred = []
        
        with tf.device('/device:XLA_GPU:0'):
            least_rmse = 1000
            for i in range(len(self.dataset[0])):
                (lstm_X_train, lstm_y_train) = self.make_dataset(self.scaled_data[i][0], self.scaled_data[i][1], time_step)
                (lstm_X_valid, lstm_y_valid) = self.make_dataset(self.scaled_data[i][2], self.scaled_data[i][3], time_step)

                model = self.Bi_Seq2_model(len(lstm_X_train[0][0]), time_step, y_step)

                history = self.Bi_Seq2_fit(model, lstm_X_train, lstm_y_train, lstm_X_valid, lstm_y_valid, time_step, patience, epochs)

                pred_temp = model.predict(lstm_X_valid)

                rmse = 0

                for j in range(y_step):
                    pred.append(pred_temp)
                    mat = self.calculate_metrics(lstm_y_valid[:, j], pred_temp[:, j])
                    print(mat)
                    rmse = rmse + mat['rmse'][0]

                    plt.plot(lstm_y_valid[:, j], label = 'actual')
                    plt.plot(pred_temp[:, j], label = 'prediction')
                        # print(self.calculate_metrics(self.inverse_min_max_scaling(self.scaler_y[i], lstm_y_valid[:, j]), self.inverse_min_max_scaling(self.scaler_y[i], pred_temp[:, j])))
                        # plt.plot(self.inverse_min_max_scaling(self.scaler_y[i], lstm_y_valid[:, j]), label = 'actual')
                        # plt.plot(self.inverse_min_max_scaling(self.scaler_y[i], pred_temp[:, j]), label = 'prediction')
                    plt.legend()
                    plt.show()

                rmse = rmse / y_step

                if rmse < least_rmse:
                    least_rmse = rmse
                    best_time_step = time_step
                    best_model = model
        
        if is_final == 1:
            best_model.save(f'{my_type}_{self.code}_model_final.h5')
        else:
            best_model.save(f'{my_type}_{self.code}_model.h5')
    
    def eval_Bi_Seq2(self, df, my_type): # 모델 평가를 위한 예측값을 도출하는 함수
        time_step = 30
        
        model = tf.keras.models.load_model(f'{my_type}_{self.code}_model.h5')
        
        new_df = df[self.type_dict[my_type]]
        (lstm_X_test, lstm_y_test) = self.make_dataset(new_df[0][0], new_df[0][1], time_step)
        
        pred = model.predict(lstm_X_test)
        
        return pred
        
    def pred_Bi_Seq2(self, my_type): # 실제 예측을 위한 함수
        
        df = self.dataset
        
        time_step = 30
        
        model = tf.keras.models.load_model(f'{my_type}_{self.code}_model_final.h5')
        new_df = df[1]
        lstm_X_test = pd.concat([new_df[0][0].iloc[-time_step:], new_df[0][1].iloc[-time_step:]], axis=1)
        (lstm_X_temp, lstm_y_temp) = self.make_dataset(new_df[0][0], new_df[0][1], time_step)

        lstm_X_test = np.array([lstm_X_test])

        return model.predict(lstm_X_test)
    
    def eval_model(self, pre, trend_pred, seasonal_pred, resid_pred, df): #pre: instance of preprocessing class
        
        y_step = 3
        lag = 15
        time_step = 30
        
        trend_pred = trend_pred[time_step - lag - 1:]
        
        y_pred = np.concatenate([trend_pred, seasonal_pred, resid_pred], axis = 2)
        
        y_pred = y_pred.transpose((1,2,0))
        
        for i in range(len(y_pred)):
            y_pred[i] = pre.inverse_minmax(y_pred[i])
        #y_pred = np.transpose(np.sum(y_pred, axis=2))
        y_pred = np.transpose(np.sum(y_pred, axis = 1))
        
        y_real_trend = np.array(df[0][0][1])
        y_real_seasonal = np.array(df[1][0][1])
        y_real_resid = np.array(df[2][0][1])
        
        y_real = np.concatenate([y_real_trend, y_real_seasonal, y_real_resid], axis = 1)
        for i in range(len(y_real)):
            y_real[i] = pre.inverse_minmax(y_real[i])
        y_real = np.transpose(np.sum(y_real, axis = 1))
        
        y_real = y_real[time_step + y_step - 1:]
        
        eval_mat = []
        
        for i in range(y_step):
            eval_mat.append(self.calculate_metrics(y_pred[:,i], y_real))
            
            plt.plot(y_real, label = 'actual')
            plt.plot(y_pred[:,i], label = 'prediction')
            plt.show()
            
        return eval_mat
    
    def save_prediction(self, trend_pred, seasonal_pred, resid_pred):
        pd.DataFrame(trend_pred.reshape(3,1)).to_csv(path_or_buf=f'./prediction_data/{self.code}_trend_pred.csv')
        pd.DataFrame(seasonal_pred.reshape(3,1)).to_csv(path_or_buf=f'./prediction_data/{self.code}_seasonal_pred.csv')
        pd.DataFrame(residual_pred.reshape(3,1)).to_csv(path_or_buf=f'./prediction_data/{self.code}_residual_pred.csv')
    

In [ ]:
# 이거 돌리면 된다~~~
#code_list = ["1018680", "1018640", "1018662", "1018683"]
code_list = ["1018662"]
for code in code_list:
    pre = Preprocessing_minmax(code)
    pre.merging()
    pre.do_blocking()
    pre.do_minmax_and_pca()
    full_data = pre.pca_full_data
    full_data_test = pre.pca_full_data_test
    full_data_final_test = pre.pca_full_data_final_test
    model = predict_model(full_data_final_test, code)
    model.fit_VAR_model("trend", is_final = 1)
    model.fit_Bi_Seq2("seasonal", is_final = 1)
    model.fit_Bi_Seq2("residual", is_final = 1)

for code in code_list:
    trend_pred = model.pred_VAR_model("trend")
    seasonal_lstm_pred = model.pred_Bi_Seq2("seasonal")
    residual_lstm_pred = model.pred_Bi_Seq2("residual")
    #seasonal_anfis_pred = model.pred_ANFIS_model("seasonal")
    #residual_anfis_pred = model.pred_ANFIS_model("residual")
    ## 적당히 seasonal, residual prediction 부분 합치기
    #save_prediction(trend_pred, seasonal_pred, resid_pred)

403
TREND SPLIT 0: SMAPE avg 0.005845939808003341, RMSE avg 8.93686890249827e-06


In [12]:
physical_devices = tf.config.list_physical_devices('GPU')

In [13]:
#physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6271030953856516466
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11740496968915732777
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9289073193153464267
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 426377216
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 16643419242364102664
physical_device_desc: "device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0"
]


2023-10-22 07:48:46.358263: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.72GiB deviceMemoryBandwidth: 836.37GiB/s
2023-10-22 07:48:46.358495: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-10-22 07:48:46.358643: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-10-22 07:48:46.358704: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-10-22 07:48:46.358753: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so.10
2023-10-22 07:48:46.358800: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcusolv